# Ajustar un modelo pre-entrenado para la clasifición de textos

Para hacer el algoritmo supervisado se usará un modelo pre-entrenado, en concreto BERT, pues es uno de los mas usados para trabajar con textos en ingles.

Tambien se hará un Fine-tuning para usar el modelo pre-entrenado y entrenarlo sobre el dataset.


Instalamos las librerías que vamos a necesitar:

# Cargando Librerias

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=4e99225ae79d7e7ecb3e01bd3d1a9ddf89141129e805e76531428b6263636710
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 12.4 GB  |     Proc size: 94.2 MB
GPU RAM Free: 15101MB | Used: 0MB | Util   0% | Total     15360MB


In [ ]:
!pip install transformers datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.utils import pad_sequences 
import tensorflow as tf 
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# asegurar que los experimentos se pueden reproducir
tf.random.set_seed(42)

tf.__version__

'2.12.0'

# Cargando Datos



In [ ]:
from google.colab import drive
from datasets import load_dataset, DatasetDict
# mount your google drive
drive.mount('/content/drive')
PATH = "/content/drive/My Drive/NLP/Proyecto/"
data_files = {"train": PATH+"data_train.csv"}

# probar cogiendo un 6%, coger train como 0.8 y el test en 0..2, despues dividir el test en 0.5
aux = load_dataset("csv", data_files=data_files, split = "train[:6%]")

dict_com = DatasetDict()
dict_com['train'] = aux
dict_com

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


DatasetDict({
    train: Dataset({
        features: ['Title', 'Review', 'Polarity', 'Country', 'Type'],
        num_rows: 15102
    })
})

# Codificación de las labels


Entrenamos un codificador con los valores del campo label. Lo que va a hacer es asignar a cada label un valor numérico:

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(dict_com['train']['Polarity'])
TARGET_LABELS = list(le.classes_)
NUM_LABELS = len(TARGET_LABELS)
NUM_LABELS

5

In [ ]:
y = le.transform(dict_com['train']['Polarity'])

Borramos el campo **file**, **label** y **label_num**.
Volvemos a añadir el campo **label** pero ahora con los valores ya transformados para que sean desde 0 a 4

In [ ]:
cleaned_dict = dict_com.remove_columns(['Title', 'Country', 'Type', 'Polarity'])
cleaned_dict

DatasetDict({
    train: Dataset({
        features: ['Review'],
        num_rows: 15102
    })
})

El dataset contiene 2015 instancias para training.

**Debemos añadir de nuevo label pero ahora con los valores codificados (es decir, los valores que están en y)**

In [ ]:
cleaned_dict['train'] = cleaned_dict['train'].add_column('label', y)
cleaned_dict

DatasetDict({
    train: Dataset({
        features: ['Review', 'label'],
        num_rows: 15102
    })
})

# Crear un split para validación. 
Como el dataset únicamente proporciona un split, vamos a tomar el 10% del training como conjunto de validación. 

voy a tomar el 30% del training para validation-test. **Despues se cambia por el conjunto test que proporcionen**

In [ ]:
aux = cleaned_dict['train'].train_test_split(test_size=0.2)
print(aux)
cleaned_dict['train']=aux['train']

# ahora vuelvo a dividir el aux en 50% para validation y test
aux = aux['test'].train_test_split(test_size=0.5)
cleaned_dict['validation'] = aux['train']
cleaned_dict['test'] = aux['test']
del(aux)

DatasetDict({
    train: Dataset({
        features: ['Review', 'label'],
        num_rows: 12081
    })
    test: Dataset({
        features: ['Review', 'label'],
        num_rows: 3021
    })
})


In [ ]:
cleaned_dict

DatasetDict({
    train: Dataset({
        features: ['Review', 'label'],
        num_rows: 12081
    })
    validation: Dataset({
        features: ['Review', 'label'],
        num_rows: 1510
    })
    test: Dataset({
        features: ['Review', 'label'],
        num_rows: 1511
    })
})

### Tokenización

Debemos preparar los textos para que tener el formato necesario para la entrada del transformer. Para ello deberemos utilizar el tokenizador asociado al modelo que vayamos a utilizar. 
Algunos modelos ya tienen clases predefinida para su tokenizador y para tareas específias. Por ejemplo, para cargar el tokenizador de **BERT** es posible utilizar la clase **BertTokenizer**. HuggingFace proporciona una clase que te permite cargar cualquier tokenizador, indicando simplemente el nombre del modelo. Esta clase es **AutoTokenizer**.

En este notebook, vamos a utilizar el modelo **bert-base-uncased**. 

In [ ]:
from transformers import AutoTokenizer
model_name="xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Vamos a aplicar el tokenizador a un texto:

In [ ]:
sentence = cleaned_dict['train'][100]['Review']
print(sentence)
print()
encoding = tokenizer(sentence)
encoding

La verdad es que fuimos 2 veces ya que la 1º estaba cerrado porque estaban de duelo nacional pero al otro dia volvimos y nos sacamos unas hermosas fotos. Esta bien resguardada y es segura para ir tanto en la mañana como en la noche.imperdible la foto del che.lotita



{'input_ids': [0, 239, 36370, 198, 41, 44625, 840, 116, 16374, 151, 41, 21, 106, 31, 18724, 209929, 3757, 82433, 8, 115, 8242, 8156, 1788, 144, 9641, 879, 2741, 75022, 7, 113, 951, 30823, 7861, 220, 7, 206291, 7, 12265, 5, 5685, 1806, 3332, 32116, 1090, 113, 198, 54426, 121, 193, 4104, 22, 21, 55371, 533, 22, 21, 41647, 5, 142864, 428, 2661, 21, 2011, 146, 290, 5, 12631, 913, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Necesitamos conocer la longitud máximas de las oraciones en el training. En función de esta longitud máxima, podemos establecer el parámetro **MAX_LENGTH** que usaremos para representar las oraciones de nuestro dataset. Todas las oraciones tendrán la misma longitud. 
Si la longitud máxima es superior a 512 tokens, la limitaremos a 512, porque el tamaño máximo que acepta el modelo BERT es 512.  

In [ ]:
MAX_LENGTH= max([len(tokenizer(text).input_ids) for text in cleaned_dict['train']['Review']])
print("La longitud máxima de la secuencia es: ", MAX_LENGTH)

#Cambiar 512 por 150

MAX_LENGTH = min(150, MAX_LENGTH)
print("max_length", MAX_LENGTH)


Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


La longitud máxima de la secuencia es:  2189
max_length 150


## Tokenizando los datos



In [ ]:
def tokenize(examples):
    return tokenizer(examples["Review"], padding="max_length", truncation=True, max_length=MAX_LENGTH)


data_encodings=cleaned_dict.map(tokenize, batched=True)
data_encodings

Map:   0%|          | 0/12081 [00:00<?, ? examples/s]

Map:   0%|          | 0/1510 [00:00<?, ? examples/s]

Map:   0%|          | 0/1511 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 12081
    })
    validation: Dataset({
        features: ['Review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1510
    })
    test: Dataset({
        features: ['Review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1511
    })
})

# Modelo 

Una vez preparados los textos. Se entrenara el modelo. Se usaran un modelo de Bert
  


In [ ]:
from transformers import AutoModelForSequenceClassification

#TARGET_LABELS = dict_dataset['train'].features['label'].names
NUM_LABELS = len(TARGET_LABELS)

print('TARGET_LABELS:', TARGET_LABELS, 'num_labels:', NUM_LABELS )

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS) 

TARGET_LABELS: [1, 2, 3, 4, 5] num_labels: 5


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Los vectores finales del modelo pre-entrenado BERT es sustituido por nuevo encabezado para la clasificación que ha sido inicializado aleatoriamente. Durante el entrenamiento, este vectores se ajustan para la tarea de clasificación, transfiriéndole el conocimiento del modelo pre-entrenado. entrenado.

#### Hyperparameteres

Necesitamos definir los hiperparámetros que se emplearán durante el proceso de entrenamiento. Algunos de estos hiperpárametros son el número de epochs para el entrenamiento, el tamaño del lote (batch), learning rate, etc.  Puedes experimentar con estos hiperparámetros para encontrar la configuración óptima para tu tarea. 

En nuestro caso, trabajaremos con los hipérparametros por defecto. Necesitamos crear un objeto de la clase TrainingArguments, que incluye todos los hiperparámetros (ya inicializados con valores por defecto) que puedes ajustar. 

In [ ]:
from transformers import TrainingArguments
args = TrainingArguments(output_dir="./outputs")
# args

Vamos a modificar algunos de ellos, por ejemplo, el tamaño del batch tanto para training como para evaluación (de 8 a 32). Además, vamos a especificar la estrategia de evaluación a epoch para que nos proporcione los resultados después de cada epoch. 


In [ ]:
args.evaluation_strategy="epoch"
args.per_device_train_batch_size = 16
args.per_device_eval_batch_size = 16


#### Métricas

También tenemos que definir el conjunto de métricas que se utilizaran para evaluar el modelo sobre el conjunto de validación. Este conjunto de métricas depende de cada tarea. En el caso de la clasificación de textos, además del accuracy, es interesante conocer la precisión, recall y f1. 

Vamos a definir una función que compute estas métricas. 


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """recibe un lote prediciones inferidas por el modelo. """
    y_true = pred.label_ids # son las labels en el gold standard
    y_pred = pred.predictions.argmax(-1) # pred.predictions devuelve una lista con las predicciones
                                        # para casda clase. Debemos quedarnos con la de mayor probabilidad.

    # como son varias clases, utilizaremos la macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#### Trainer

Ya estamos listos para entrenar el modelo. Para ello, Pytorch ya nos proporciona una clase **Trainer**,  que está optimizada para entrenar transformers, y que además nos va a ahorrar mucho trabajo (no será necesario escribir el ciclo de entrenamiento por epochs y calcular las métricas sobre el conjunto de validación). 

Para crear este objeto Trainer, deberemos pasarle el modelo, los argumentos, la función para compuar las métricas, y el conjunto de entrenamiento y validación.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,            # modelo que será ajustado
    args = args,     # hiperparámetros
    train_dataset=data_encodings['train'], # conjunto training
    eval_dataset=data_encodings['validation'],   # conjunto de validación
    compute_metrics=compute_metrics,    # función para computar las métricas
)

Por fin, podemos entrenar para ajustar el modelo a la tarea de clasificación de textos: 

In [ ]:
#import torch
#torch.cuda.empty_cache()
#torch.cuda.memory_summary(device=None, abbreviated=False)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


Una vez finalizado el entrenamiento, es posible evaluar el modelo final sobre el conjunto de datos de validación. Las métricas obtenidas son: 

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.0450819730758667,
 'eval_accuracy': 0.6309050772626932,
 'eval_f1': 0.15473741201949107,
 'eval_precision': 0.12618101545253863,
 'eval_recall': 0.2,
 'eval_runtime': 68.6449,
 'eval_samples_per_second': 32.996,
 'eval_steps_per_second': 2.069,
 'epoch': 3.0}

In [ ]:
trainer.save_model( PATH + "berta")

In [ ]:
trainer_model = AutoModelForSequenceClassification.from_pretrained(PATH + "berta", num_labels=NUM_LABELS) 

## Evaluación

Por último, vamos a utilizar el modelo para predecir las clases para textos que no han sido utilizados durane el entrenamiento. Es decir, vamos a aplicar el modelo sobre el conjunto test.

La siguiente función recibe un texto y devuelve la clase inferida por el modelo. La función codifica el texto usando el tokenizador y el modelo es aplicado sobre esta codificación. Sobre la salida del modelo, aplicaremos una función softmax, que calcule la probabilidad de cada clase. Finalmente, devolvemos la clase con mayor probabilidad (usaremos la función **argmax**)
 .


In [ ]:
def get_prediction(text):
    # prepara el texto, aplicamos la misma tokenización que la utilizada en el training
    inputs = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    # aplicamos el modelo
    outputs = model(**inputs)
    # obtenemos la probabilidad para cada clase
    probs = outputs[0].softmax(1)
    # argmax nos devuelve la clase con mayor probabilidad.
    # argmax devuelve un tensor. Debemos devolver su valor asociado 
    return probs.argmax().item()

Vamos a aplicar la función **get_prediction** al conjunto de evaluación:

In [ ]:
y_pred=[get_prediction(text) for text in cleaned_dict['test']['Review']]
y_test = cleaned_dict['test']['label']

Mostramos los resultados finales. 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00        22
           3       0.00      0.00      0.00        63
           4       0.63      1.00      0.77       158

    accuracy                           0.63       252
   macro avg       0.13      0.20      0.15       252
weighted avg       0.39      0.63      0.48       252



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0, 1, 2, 3, 4]),
 [Text(0, 0, '0'),
  Text(1, 0, '1'),
  Text(2, 0, '2'),
  Text(3, 0, '3'),
  Text(4, 0, '4')])

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues,values_format='g')
plt.xticks(rotation=90)

Al usar el modelo pre-entrenado se obtiene una buena clasificación de los datos